In [ ]:
using ITensors 
using Plots
# using Measurements   
# using DelimitedFiles  

In [ ]:
L = 20;
N = 1;
G² = 1.5;
m = 0.5;
sites = siteinds("Fermion", L);

In [ ]:
function DMRG(l)
    ampo = OpSum();
    for j in 1:l-N
        ampo -= 1im,"C",j,"Cdag",j+N
        ampo += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:l-N
        ampo += (G²/2),"C * Cdag",j
    end
    for j in 1:l
        ampo += (m*(-1)^j),"C * Cdag",j
    end
    H = MPO(ampo,sites[1:l]);
    ψ₀ = randomMPS(sites[1:l],100);
    sweep = Sweeps([
         "maxdim" "mindim" "cutoff" 
          10       2       1e-16    
          30       10      1e-16    
          50       20      1e-16    
          1000     30      1e-18    
          1024     50      1e-18
         ])
    e,ψ = dmrg(H,ψ₀,sweep,outputlevel=1)
    return e,ψ
end

In [ ]:
mps_vector = Vector{MPS}()
for j in 4:2:L
    println("j = ",j)
    e₀, ψ₀ = DMRG(j)
    push!(mps_vector, ψ₀)
end


In [ ]:
inner_v = Vector{ComplexF64}()
norm_v = Vector{ComplexF64}()

for j in 2:length(mps_vector)
    large_state = mps_vector[j]
    small_state = mps_vector[j-1]
    len = length(large_state)
    v1 = ITensor(mps_vector[j][len-1])
    v2 = ITensor(mps_vector[j][len])
    v = ITensor(1.0)
    for k in 1:16
        v1[k] = 1/4
    end
    for k in 1:4
        v2[k] = 1/2
    end
    for k in 1:(len-2)
        v = v*dag(large_state[k])*small_state[k]
    end
    v = v*v1*large_state[len-1]
    v = v*v2*large_state[len]
    push!(inner_v,inner(v,v))
    push!(norm_v,norm(v))
end
    

In [ ]:
res_v = Vector{ComplexF64}()
norm_v = Vector{ComplexF64}()
mps_vector = Vector{MPS}()
for k in 4:2:L
    eₗ, ψₗ = DMRG(k)
    eₛ, ψₛ = DMRG(k-2)
    v1 = ITensor(ψₗ[k-1])
    v2 = ITensor(ψₗ[k])
    v = ITensor(1.0)
    println(norm(v))
    for j in 1:16
        v1[j] = 1/4
    end
    for j in 1:4
        v2[j] = 1/2
    end
    println(norm(v1))
    println(norm(v2))
    for j in 1:k-2
        v = v*dag(ψₗ[j])*ψₛ[j]
    end
    v = v*v1*ψₗ[k-1]
    v = v*v2*ψₗ[k]
    println(norm(v))
    push!(res_v,inner(v,v))
    push!(norm_v,norm(v))
    v = ITensor(0.0)
end;

In [ ]:
site_number = [x for x in 1:length(res_v)]
res_v2 = real(res_v)
norm_v2 = real(norm_v)
scatter(site_number, 
    norm_v2, 
#     xlim=(-1,20),
#     xticks=((1:2:20)), 
#     ylim=(-0.1,1), 
#     yticks=((-0.1:0.1:0.3)),   
    markersize=2,
    xlabel="Site Number",
    ylabel="Inner Product",
    title="Overlap")
# savefig("GN_m07_50.svg")

In [ ]:

result_res2 = Vector{ComplexF64}()
result_norm2 = Vector{ComplexF64}()
for j in 4:2:10
#     ψₗ = MPS(ψ[1:j])/norm(MPS(ψ[1:j]))
#     ψₛ = MPS(ψ[1:j-2])/norm(MPS(ψ[1:j-2]))
    ψₗ = DMRG(j)
    ψₛ = DMRG(j-2)
    v1 = ITensor(ψ[j])/norm(ITensor(ψ[j]))
    for k in 1:size(v1)[1]
        v1[k] = 1/sqrt(2)
    end
    v2 = ITensor(ψ[j-1])
    for k in 1:size(v2)[1]
        v2[k] = 1/sqrt(2)
    end
    
    v = ITensor(1.0)
    for k in 1:j-2
        v = v*dag(ψₛ[k])*ψₗ[k]
    end
    v = v*v1*ψₗ[j-1]
    v = v*v2*ψₗ[j]
#     return inner(v,v), norm(v)
    push!(result_res2, inner(v,v))
    push!(result_norm2, norm(v))  
end
    
    